<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/LinearProgramming/LP_Problem_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 11: During the next three months, Steelco faces the following demands for steel: 100 tons (month 1); 200 tons (month 2); 50 tons (month 3). During any month, a worker can produce up to 15 tons of steel. Each worker is paid $\$ 5,000$ per month. Workers can be hired or fired at a cost of $\$ 3,000$ per worker fired and $\$ 4,000$ per worker hired (it takes 0 time to hire a worker). The cost of holding a ton of steel in inventory for one month is $\$ 100$. Demand may be backlogged at a cost of $\$ 70$ per ton month. That is, if 1 ton of month 1 demand is met during month 3 , then a backlogging cost of $\$ 140$ is incurred. At the beginning of month 1, Steelco has 8 workers. During any month, at most 2 workers can be hired. All demand must be met by the end of month 3. The raw material used to produce a ton of steel costs $\$ 300$. Formulate an LP to minimize Steelco's costs.

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 62.5 MB/s eta 0:00:00


In [5]:
import pulp

# Parameters #
months = [1, 2, 3]
demand = {1: 100, 2: 200, 3: 50}
prod_per_worker = 15
worker_cost = 5000
hire_cost = 4000
fire_cost = 3000
inv_cost = 100
backlog_cost = 70
material_cost = 300
initial_workers = 8
max_hire = 2

# LP model #
model = pulp.LpProblem("Steelco_Production_Planning", pulp.LpMinimize)

# Decision variables #
x = pulp.LpVariable.dicts("Production", months, lowBound=0)
I = pulp.LpVariable.dicts("Inventory", months, lowBound=0)
B = pulp.LpVariable.dicts("Backlog", months, lowBound=0)
W = pulp.LpVariable.dicts("Workers", months, lowBound=0, cat = "Integer")
H = pulp.LpVariable.dicts("Hire", months, lowBound=0, cat = "Integer")
F = pulp.LpVariable.dicts("Fire", months, lowBound=0, cat = "Integer")

# Objective function #
model += pulp.lpSum([
    worker_cost * W[t] +
    hire_cost * H[t] +
    fire_cost * F[t] +
    material_cost * x[t] +
    inv_cost * I[t] +
    backlog_cost * B[t]
    for t in months
])

# Constraints #
for t in months:
    # Production capacity #
    model += x[t] <= prod_per_worker * W[t]

    # Worker balance #
    if t == 1:
        model += W[t] == initial_workers + H[t] - F[t]
    else:
        model += W[t] == W[t-1] + H[t] - F[t]

    # Hiring limit #
    model += H[t] <= max_hire

    # Inventory/backlog balance #
    if t == 1:
        model += I[t] - B[t] == x[t] - demand[t]
    else:
        model += I[t] - B[t] == I[t-1] - B[t-1] + x[t] - demand[t]

# Final conditions: no inventory or backlog left at end #
model += I[3] == 0
model += B[3] == 0

# Solve #
model.solve(pulp.PULP_CBC_CMD(msg=0))

# Print results #
print("Status:", pulp.LpStatus[model.status])
print("Total Cost =", pulp.value(model.objective))
for t in months:
    print(f"Month {t}:")
    print("  Workers =", W[t].varValue)
    print("  Hire =", H[t].varValue, " Fire =", F[t].varValue)
    print("  Production =", x[t].varValue)
    print("  Inventory =", I[t].varValue)
    print("  Backlog =", B[t].varValue)


Status: Optimal
Total Cost = 230900.0
Month 1:
  Workers = 8.0
  Hire = 0.0  Fire = 0.0
  Production = 110.0
  Inventory = 10.0
  Backlog = 0.0
Month 2:
  Workers = 8.0
  Hire = 0.0  Fire = 0.0
  Production = 120.0
  Inventory = 0.0
  Backlog = 70.0
Month 3:
  Workers = 8.0
  Hire = 0.0  Fire = 0.0
  Production = 120.0
  Inventory = 0.0
  Backlog = 0.0
